In [1]:
import numpy as np
from astropy.table import Table
catalog = Table.read('/global/cscratch1/sd/jharno/IA_Infusion/GalCat_tomo1_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat',
                     format='ascii')  # descriptions=['ra_deg', 'dec_deg', 'shear1', 'shear2', 'weight', 'z_true', 'delta', 's11', 's22', 's12'])
#epsilons = Table.read('/global/cscratch1/sd/lhior/epsilon.dat', format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
epsilons = Table.read('/global/cscratch1/sd/jharno/epsilon.dat', format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']


gamma1 = np.asarray(catalog['col3'])
gamma2 = np.asarray(catalog['col4'])
e1_NLA = np.asarray(epsilons['e1_NLA'])
e2_NLA = np.asarray(epsilons['e2_NLA'])
e1_TT = np.asarray(epsilons['e1_TT'])
e2_TT = np.asarray(epsilons['e2_TT'])
e1_noise = np.asarray(epsilons['e1_noise'])
e2_noise = np.asarray(epsilons['e2_noise'])
delta = np.asarray(catalog['col7'])

FileNotFoundError: [Errno 2] No such file or directory: '/global/cscratch1/sd/jharno/epsilon.dat'

We use the constants for different IA models as inputs

In [7]:
A_IA = 1
b_TA = 0
##I think I'm forgetting something here

The intensity of the NLA and delta-NLA models can be chosen by tweaking the numbers above, this can probably be changed for some calculation involving weights given to each model but that's up for discussion, for now, we setup for the observed epsilons with and without noise, using the provided parameters

In [18]:
epsilon_IA = np.empty(e1.shape, dtype = complex)
epsilon_IA.real = A_IA * e1  
epsilon_IA.imag = A_IA * e2
epsilon_random = np.empty(e1.shape, dtype = complex)
epsilon_random.real = e1_noise
epsilon_random.imag = e2_noise
g = np.empty(gamma1.shape, dtype=complex)
g.real = gamma1
g.imag = gamma2

And finally, we calculate the observed epsilons using the summation as in equation 11 of the paper

In [19]:
epsilon_IA_delta = epsilon_IA * (1 + b_TA * delta) #Placeholder since current deltas are not final, simplifies to epsilon_IA for b_TA = 0
epsilon_int = (epsilon_IA_delta + epsilon_random)/(1 + epsilon_IA_delta * epsilon_random)
epsilon_obs = (epsilon_int + g)/(1 + epsilon_int * g)
epsilon_obs_no_noise = (epsilon_IA_delta + g)/(1 + epsilon_IA_delta * g)